In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [0]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [0]:
sns.set_style('whitegrid')
plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

In [0]:
df = pd.read_csv('/Workspace/Users/nagahamasami@gmail.com/creditas-case/data/raw/dataset.csv')
df.head()

In [0]:
df.tail()

In [0]:
description = pd.read_csv('/Workspace/Users/nagahamasami@gmail.com/creditas-case/data/raw/description.csv')
description

In [0]:
# Visão geral do dataset
df.info()

In [0]:
# Estatísticas descritivas gerais
df.describe().round(2)

# Valores Faltantes

In [0]:
# Análise de valores faltantes
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100
missing_df = pd.DataFrame({'Contagem': missing_values, 'Porcentagem': missing_percentage})
missing_df = missing_df[missing_df['Contagem'] > 0].sort_values(by='Porcentagem', ascending=False)
print(missing_df)

In [0]:
plt.figure(figsize=(12, 8))
sns.heatmap(df.isnull(), cbar=False, cmap='viridis')
plt.title('Mapa de Calor dos Valores Faltantes')
plt.show()

In [0]:
plt.figure(figsize=(15, 7))
missing_df['Porcentagem'].plot(kind='bar')
plt.title('Porcentagem dos Valores Faltantes por Coluna')
plt.ylabel('(%)')
plt.xlabel('Colunas')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

> Dada a grande falta de valores para loan_term,  marital_status e utm_term, estas colunas podem ser sumariamente descartadas do treinamento do modelo.

> A imputação de dados para essas colunas adicionaria ruído e vieses para o conjunto.

# Sanity Check

In [0]:
min_age, max_age = df['age'].min(), df['age'].max()
print(f"Faixa Etária: Mín={min_age:.1f}, Máx={max_age:.1f}")
if min_age < 18 or max_age > 120:
    print("Idade fora da faixa esperada de 18-120.")

In [0]:
financial_cols = ['monthly_income', 'collateral_value', 'loan_amount', 'monthly_payment', 'collateral_debt']
for col in financial_cols:
    negative_count = (df[col] < 0).sum()
    if negative_count > 0:
        print(f"A coluna '{col}' contém {negative_count} valores negativos.")
    else:
        print(f"'{col}' não contém valores negativos.")

In [0]:
min_year, max_year = df['auto_year'].min(), df['auto_year'].max()
current_year = pd.to_datetime('today').year
print(f"\nFaixa de Ano do Veículo: Mín={min_year:.0f}, Máx={max_year:.0f}")
if max_year > current_year + 1:
    print(f"Ano do veículo '{max_year}' está no futuro.")
if min_year < 1950:
    print(f"Ano do veículo '{min_year}' muito antigo.")

In [0]:
inconsistent_funnel = df[(df['sent_to_analysis'] == 1) & (df['pre_approved'] == 0)]
if not inconsistent_funnel.empty:
    print(f"Encontrados {len(inconsistent_funnel)} registros em que o cliente foi enviado à análise mas NÃO estava pré-aprovado.")
else:
    print("Lógica do Funil (sent_to_analysis requer pre_approved).")

In [0]:
impossible_payment = df[df['monthly_payment'] > df['monthly_income']]
if not impossible_payment.empty:
    print(f"Encontrados {len(impossible_payment)} registros em que monthly_payment > monthly_income.")
else:
    print("Lógica Financeira (pagamento < renda): OK.")

In [0]:
impossible_collateral = df[df['collateral_debt'] > df['collateral_value']]
if not impossible_collateral.empty:
    print(f"Encontrados {len(impossible_collateral)} registros em que collateral_debt > collateral_value.")
else:
    print("Lógica de Garantia (dívida <= valor): OK.")

In [0]:
categorical_cols_check = ['state', 'channel', 'gender', 'education_level']
for col in categorical_cols_check:
    print(f"\nValores únicos para '{col}':")
    print(df[col].dropna().unique())

In [0]:
is_id_unique = df['id'].is_unique
if is_id_unique:
    print("Ok.")
else:
    duplicate_id_count = df['id'].duplicated().sum()
    print(f"Encontrados {duplicate_id_count} IDs duplicados.")


# Análise Univariada

## Variáveis Numéricas

In [0]:
cols_to_plot = [
    {'col': 'age', 'title': 'Distribuição da Idade', 'xlabel': 'Idade'},
    {'col': 'monthly_income', 'title': 'Distribuição da Renda Mensal', 'xlabel': 'Renda Mensal'},
    {'col': 'collateral_value', 'title': 'Distribuição do Valor do Veículo (Garantia)', 'xlabel': 'Valor do Veículo'},
    {'col': 'loan_amount', 'title': 'Distribuição do Valor do Empréstimo', 'xlabel': 'Valor do Empréstimo'},
    {'col': 'collateral_debt', 'title': 'Distribuição da Dívida do Veículo', 'xlabel': 'Dívida do Veículo'},
    {'col': 'monthly_payment', 'title': 'Distribuição do Pagamento Mensal', 'xlabel': 'Pagamento Mensal'}
]

num_rows = len(cols_to_plot)
fig, axes = plt.subplots(nrows=num_rows, ncols=2, figsize=(14, num_rows * 4))

fig.suptitle('Análise Univariada das Variáveis Numéricas', fontsize=20, y=1.02)

for i, var_info in enumerate(cols_to_plot):
    col_name = var_info['col']
    title = var_info['title']
    xlabel = var_info['xlabel']

    sns.histplot(data=df, x=col_name, kde=True, bins=30, ax=axes[i, 0], color='skyblue')
    axes[i, 0].set_title(f'{title} - Histograma')
    axes[i, 0].set_xlabel(xlabel)
    axes[i, 0].set_ylabel('Frequência')

    sns.boxplot(data=df, x=col_name, ax=axes[i, 1], showfliers=False, color='lightgreen')
    axes[i, 1].set_title(f'{title} - Boxplot')
    axes[i, 1].set_xlabel(xlabel)

plt.tight_layout(rect=[0, 0, 1, 1])
plt.show()

## Variáveis Categóricas

In [0]:
horizontal_cols = {
    'city': 'Top 10 Cidades',
    'state': 'Top 10 Estados',
    'auto_brand': 'Top 10 Marcas de Veículos',
    'auto_model': 'Top 10 Modelos de Veículos',
    'auto_year': 'Top 10 Anos de Veículos',
    'education_level': 'Distribuição do Nível Educacional'
}

vertical_cols = {
    'verified_restriction': 'Restrição Verificada',
    'dishonored_checks': 'Cheques sem Fundo',
    'expired_debts': 'Dívidas Expiradas',
    'banking_debts': 'Dívidas Bancárias',
    'commercial_debts': 'Dívidas Comerciais',
    'protests': 'Protestos',
    'informed_restriction': 'Restrição Informada',
    'pre_approved': 'Pré-Aprovados',
    'form_completed': 'Formulário Completo',
    'sent_to_analysis': 'Enviado para Análise',
    'channel': 'Canal de Aquisição',
    'landing_page_product': 'Produto da Landing Page',
    'gender': 'Gênero',
    'utm_term': 'Tipo de Dispositivo (UTM)'
}

total_plots = len(horizontal_cols) + len(vertical_cols)
num_cols = 3
num_rows = math.ceil(total_plots / num_cols)

fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(20, num_rows * 5))
axes = axes.flatten()

fig.suptitle('Análise Univariada das Variáveis Categóricas', fontsize=24, y=1.03)

current_ax_index = 0
for col, title in horizontal_cols.items():
    ax = axes[current_ax_index]
    top_10 = df[col].value_counts().nlargest(10).index
    sns.countplot(y=df[col], order=top_10, ax=ax, palette='viridis')
    ax.set_title(title, fontsize=14)
    ax.set_xlabel('Contagem')
    ax.set_ylabel('')
    current_ax_index += 1

for col, title in vertical_cols.items():
    ax = axes[current_ax_index]
    order = df[col].value_counts().index
    sns.countplot(x=df[col], order=order, ax=ax, palette='viridis')
    ax.set_title(title, fontsize=14)
    ax.set_xlabel('')
    ax.set_ylabel('Contagem')
    if len(order) > 3:
         ax.tick_params(axis='x', rotation=45)
    current_ax_index += 1

for i in range(current_ax_index, len(axes)):
    axes[i].set_visible(False)

plt.tight_layout(rect=[0, 0, 1, 0.99])
plt.show()

In [0]:
unique_counts = df.nunique()
print(unique_counts)

# Análise bivariada

In [0]:
total_clients = len(df)
pre_approved_count = df['pre_approved'].sum() # .sum() funciona porque True=1 e False=0
not_pre_approved_count = total_clients - pre_approved_count
pre_approved_percentage = (pre_approved_count / total_clients) * 100

print(f"Quantidade total de clientes no conjunto de dados: {total_clients}")
print(f"Quantidade de clientes Pré-Aprovados: {int(pre_approved_count)}")
print(f"Quantidade de clientes Não Pré-Aprovados: {int(not_pre_approved_count)}")
print(f"Porcentagem de clientes que são Pré-Aprovados: {pre_approved_percentage:.2f}%")

status_data = pd.DataFrame({
    'Status': ['Pré-Aprovado', 'Não Pré-Aprovado'],
    'Contagem': [pre_approved_count, not_pre_approved_count]
})

plt.figure(figsize=(8, 6))
ax = sns.barplot(x='Status', y='Contagem', data=status_data, palette=['#4CAF50', '#F44336'])

for p in ax.patches:
    ax.annotate(f'{int(p.get_height())}',
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center',
                xytext=(0, 9),
                textcoords='offset points',
                fontsize=12,
                weight='bold')

plt.title('Total vs. Clientes Pré-Aprovados (Contagens Absolutas)', fontsize=16)
plt.ylabel('Número de Clientes', fontsize=12)
plt.xlabel('Status do Cliente', fontsize=12)
plt.show()

In [0]:
df_analysis = df[df['pre_approved'] == 1].copy()

crosstab_counts = pd.crosstab(df_analysis['form_completed'], df_analysis['sent_to_analysis'])
crosstab_counts.index = ['Não Preencheu o Formulário', 'Preencheu o Formulário']
crosstab_counts.columns = ['Não Enviado para Análise', 'Enviado para Análise']

crosstab_normalized = pd.crosstab(df_analysis['form_completed'], df_analysis['sent_to_analysis'], normalize='index') * 100
crosstab_normalized.index = ['Não Preencheu o Formulário', 'Preencheu o Formulário']
crosstab_normalized.columns = ['Não Enviado para Análise (%)', 'Enviado para Análise (%)']

ax = crosstab_normalized.plot(
    kind='bar',
    stacked=True,
    figsize=(10, 7),
    colormap='viridis',
    rot=0
)

plt.title('Impacto do Preenchimento do Formulário no Envio para Análise', fontsize=16, pad=20)
plt.ylabel('Porcentagem de Clientes (%)', fontsize=12)
plt.xlabel('Status de Preenchimento do Formulário', fontsize=12)

for c in ax.containers:
    labels = [f'{w:.1f}%' if w > 0 else '' for w in c.datavalues]
    ax.bar_label(c, labels=labels, label_type='center', weight='bold', color='white')

plt.legend(title='Resultado', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [0]:
education_mapping = {
    'Analfabeto, inclusive o que, embora tenha recebido instrução, não se alfabetizou.': 'Analfabeto',
    'Até a 4ª série incompleta do ensino fundamental (antigo 1º grau ou primário), que se tenha alfabetizado sem ter freqüentado escola regular.': 'Fundamental Incompleto',
    '4ª série completa do ensino fundamental (antigo 1º grau ou primário).': 'Fundamental Incompleto',
    'Da 5ª à 8ª série do ensino fundamental (antigo 1º grau ou ginásio).': 'Fundamental Incompleto',
    'Ensino fundamental completo (antigo 1º grau ou primário e ginasial).': 'Fundamental Completo',
    'Ensino médio incompleto (antigo 2º grau, secundário ou colegial).': 'Ensino Médio Incompleto',
    'Ensino médio completo (antigo 2º grau, secundário ou colegial).': 'Ensino Médio Completo',
    'Educação superior incompleta.': 'Superior Incompleto',
    'Educação superior completa': 'Superior Completo'
}

df['education_level'] = df['education_level'].replace(education_mapping)

In [0]:
categorical_features = [
    'form_completed', 'channel', 'education_level',
    'state', 'informed_purpose', 'gender'
]

In [0]:
n_features = len(categorical_features)
n_cols = 2
n_rows = math.ceil(n_features / n_cols)

fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 6 * n_rows))
axes = axes.flatten()

for i, feature in enumerate(categorical_features):
    if df_analysis[feature].nunique() > 10:
        top_categories = df_analysis[feature].value_counts().nlargest(10).index
        plot_df = df_analysis[df_analysis[feature].isin(top_categories)]
        title = f'Taxa de Conversão pelas 10 Principais "{feature}"'
    else:
        plot_df = df_analysis
        title = f'Taxa de Conversão por "{feature}"'

    conversion_rates = (
        plot_df.groupby(feature)['sent_to_analysis']
        .mean()
        .sort_values(ascending=False)
    )

    sns.barplot(
        x=conversion_rates.index,
        y=conversion_rates.values * 100,
        palette='viridis',
        ax=axes[i]
    )

    axes[i].set_title(title, fontsize=14)
    axes[i].set_ylabel('Taxa de Envio (%)', fontsize=10)
    axes[i].set_xlabel(feature, fontsize=10)
    axes[i].tick_params(axis='x', rotation=90)

for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.subplots_adjust(hspace=0.5, wspace=0.3)
plt.show()

In [0]:
financial_flags = ['dishonored_checks', 'expired_debts', 'banking_debts', 'commercial_debts', 'protests']
flag_conversion_rates = {}
for flag in financial_flags:
    rate = df_analysis.groupby(flag)['sent_to_analysis'].mean()
    flag_conversion_rates[flag] = rate

fig, axes = plt.subplots(1, len(financial_flags), figsize=(20, 5), sharey=True)
fig.suptitle('Impacto de Indicadores Financeiros Negativos na Taxa de Conversão', fontsize=16)
for i, flag in enumerate(financial_flags):
    rates = flag_conversion_rates[flag]
    sns.barplot(x=rates.index, y=rates.values * 100, ax=axes[i], palette='viridis')
    axes[i].set_title(flag)
    axes[i].set_xlabel('Indicador (1=Sim, 0=Não)')
    if i == 0:
        axes[i].set_ylabel('Taxa de Envio para Análise (%)')
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [0]:
numerical_features = [
    'monthly_income', 'loan_amount', 'collateral_value',
    'collateral_debt', 'auto_year'
]

fig, axes = plt.subplots(1, len(numerical_features), figsize=(20, 6))

fig.suptitle('Distribuição das Variáveis Numéricas por Resultado da Análise', fontsize=18, y=1.02)

for i, feature in enumerate(numerical_features):
    sns.boxplot(
        x='sent_to_analysis',
        y=feature,
        data=df_analysis,
        palette='coolwarm',
        showfliers=False,
        ax=axes[i]
    )
    
    axes[i].set_title(f'{feature}', fontsize=14)
    axes[i].set_ylabel(feature, fontsize=12)
    axes[i].set_xlabel('Enviado para Análise\n(0=Não, 1=Sim)', fontsize=11)
    
    if feature in ['monthly_income', 'collateral_value', 'loan_amount']:
        axes[i].set_ylim(0, df_analysis[feature].quantile(0.95))

plt.tight_layout()
plt.show()

> Clientes com rendas mais altas têm maior chance de serem enviados para análise. A renda é uma métrica fundamental para avaliar a capacidade de pagamento do empréstimo.

>  Clientes que solicitam empréstimos muito altos têm menos probabilidade de avançar no processo.

> Um carro de maior valor oferece melhor garantia para o empréstimo.

> Dívidas existentes complicam o processo do empréstimo e aumentam o risco.

> Carros mais antigos têm valor menor e podem ser considerados de maior risco como garantia.

# Limpeza e pré-processamento

In [0]:
df_processed = df.copy()

In [0]:
df_processed.shape

In [0]:
df_processed = df_processed[df_processed['pre_approved'] == 1].copy()
print(f"Shape após filtrar por pre-approved': {df_processed.shape}")

In [0]:
# Removendo colunas com muitos valores nulos
cols_to_drop = ['loan_term', 'marital_status', 'utm_term']
df_processed.drop(columns=cols_to_drop, inplace=True)

In [0]:
# Imputando valores faltantes com 0
nan_imputation = {
    'collateral_debt': 0,
    'verified_restriction': 0,
    'informed_restriction': 0
}
df_processed.fillna(nan_imputation, inplace=True)

In [0]:
# Imputando colunas categóricas com "Unknown"
categorical_cols = df_processed.select_dtypes(include=['object']).columns
for col in categorical_cols:
    df_processed[col].fillna('Unknown', inplace=True)

In [0]:
print(df_processed.duplicated().sum())
df_processed.drop_duplicates(inplace=True)

In [0]:
print(df_processed['id'].duplicated().sum())

In [0]:
df_processed = df_processed.drop_duplicates(subset=['id'], keep='first')

In [0]:
df_processed.info()

In [0]:
df_processed.to_csv('/Workspace/Users/nagahamasami@gmail.com/creditas-case/data/processed/clean-dataset.csv', index=False)